<a href="https://colab.research.google.com/github/Arnes1997/product_category_classifier/blob/main/notebook/category_classifier_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Product Category Classifier

Ovaj projekat razvijen je s ciljem **automatske klasifikacije proizvoda po kategorijama** na osnovu njihovih naslova i dodatnih atributa.  
Koristi se **machine learning model** obučen na stvarnom skupu podataka sa više od 30.000 proizvoda različitih vrsta (telefoni, kamere, mikrotalasne, frižideri i dr.).

---

## 📊 Opis skupa podataka

Ulazni skup podataka (`products.csv`) sadrži sledeće kolone:

| Kolona | Opis |
|--------|------|
| **Product ID** | Jedinstveni identifikator proizvoda |
| **Product Title** | Naziv proizvoda (npr. “Samsung Galaxy A52 128GB”) |
| **Merchant ID** | Identifikator prodavca |
| **Category Label** | Kategorija proizvoda (ciljna varijabla) |
| **Product Code** | Interni kod proizvoda |
| **Number of Views** | Broj pregleda proizvoda |
| **Merchant Rating** | Ocena prodavca |
| **Listing Date** | Datum postavljanja proizvoda |

1. **Učitavanje i istraživanje podataka (Exploratory Data Analysis)**  
   - Provjera jedinstvenih vrijednosti.  
   - Provera null vrednosti.  
   - Pregled osnovnih statistika i strukture podataka.  

In [11]:
# Ucitavanje potrebne biblioteke
import pandas as pd

# Ucitavanje products.csv fajla u dataframe
df = pd.read_csv("https://raw.githubusercontent.com/Arnes1997/product_category_classifier/refs/heads/main/data/products.csv")

# Pregled strukture podataka
print("*" * 100)
print("*** Kolone i tipovi podataka ***")
print(df.info())
print("*" * 100)
print("*** Prvih 5 redova ***")
print(df.head())
print("*" * 100)

# Pregled jedinstvenih vrijednosti
print("*** Jedinstvene vrijednosti u koloni Category Label ***")
print(df[' Category Label'].value_counts())
print("*" * 100)

# Provjera da li postoje null vrijednosti u df
print("*** Null vrijednosti ***")
print(df.isnull().sum())
print("*" * 100)



****************************************************************************************************
*** Kolone i tipovi podataka ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35311 entries, 0 to 35310
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product ID       35311 non-null  int64  
 1   Product Title    35139 non-null  object 
 2   Merchant ID      35311 non-null  int64  
 3    Category Label  35267 non-null  object 
 4   _Product Code    35216 non-null  object 
 5   Number_of_Views  35297 non-null  float64
 6   Merchant Rating  35141 non-null  float64
 7    Listing Date    35252 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.2+ MB
None
****************************************************************************************************
*** Prvih 5 redova ***
   product ID                                      Product Title  Merchant ID  \
0           1             

2. **Čišćenje i priprema podataka (Data Cleaning)**  
   - Uklanjanje praznih redova.  
   - Uklanjanje nepotrbnih kolona.

In [22]:
# Zadrzava samo kolone product title i category label za treniranje modela
df = df[['Product Title', ' Category Label']]

# Ukloni redove koji imaju null u koloni 'Category Label' i 'Product Title'
df = df.dropna(subset=[' Category Label'])
df = df.dropna(subset=['Product Title'])

# Provjera da li je ociscena kolona Category Label
print("Null vrijednosti nakon ciscenja kolone Category Label: ", df[' Category Label'].isnull().sum())

# Provjera da li je ociscena kolona Product Title
print("Null vrijednosti nakon ciscenja kolone Product Title: ", df['Product Title'].isnull().sum())

Null vrijednosti nakon ciscenja kolone Category Label:  0
Null vrijednosti nakon ciscenja kolone Product Title:  0


3. **Inženjering karakteristika (Feature Engineering)**  
   Pored tekstualnih informacija iz naziva proizvoda, dodate su i numeričke osobine koje pomažu modelu da bolje razlikuje kategorije:  
   - `title_length` – dužina naslova (broj karaktera)  
   - `num_words` – broj riječi u naslovu  
   - `has_digits` – da li naslov sadrži brojeve  
   - `has_uppercase` – da li postoje riječi napisane velikim slovima  

In [23]:
# Ucitavanje potrebne biblioteke
import re

# Izračunava dužinu naslova proizvoda (broj karaktera)
df['title_length'] = df['Product Title'].apply(len)

# Izračunava broj riječi u naslovu proizvoda
df['num_words'] = df['Product Title'].apply(lambda x: len(x.split()))

# Provjerava da li naslov sadrži brojeve (0 ili 1)
df['has_digits'] = df['Product Title'].apply(lambda x: int(bool(re.search(r'\d', x))))

# Provjerava da li naslov sadrži barem jednu riječ napisanu velikim slovima
df['has_uppercase'] = df['Product Title'].apply(lambda x: int(any(w.isupper() for w in x.split())))

# Prikaz kolona
print("Kolone nakon (Feature Engineering):")
df.columns

Kolone nakon (Feature Engineering):


Index(['Product Title', ' Category Label', 'title_length', 'num_words',
       'has_digits', 'has_uppercase'],
      dtype='object')